<a href="https://colab.research.google.com/github/jhmartel/fp/blob/master/_notebooks/2023-01-15-PolynomialSubsetSums.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zero Subset Sums and Laurent Polynomials

> "(Under Construction). Here we collect some trivial remarks on using polynomials and symbolic calculators, e.g. Mathematica, SymPy, to find zero subset sums." 

- toc: false
- branch: master
- badges: false
- comments: true
- author: JHM

_Here we collect some elementary observations on zero sum problems using symbolic calculators. This post is not complete. We simply wanted to demonstrate *how* to use polynomials in the style of generating functions to compute zero sums. [-JHM]_

# Symbolic Solutions to Zero Sum Problems

The zero subset sum problem is this: 

_Given a generic subset $S\subset \bf{Z}$, decide whether there exists a finite subset $S' \subset S$ which sums to zero._

The trouble here is always sampling over the powerset $2^S$ of all subsets of $S$. It's a very large set! Is there a categorical approach to enumerating the powerset? Here enters symbolic algebra, and a simple idea: 

Given a finite subset $S \subset \mathbf{Z}$, we formally define $$p=p_S(x)=\sum_{s\in S} x^s.$$ Evidently $p_S \in \mathbf{Z}[x, x^{-1}]$ belongs to the ring of Laurent polynomials, and finitely supported since $\#(S) < +\infty$. 

_A trivial observation: the coefficient of the degree zero term $x^0=1$ in the powers $p^k$ of $p=p_S$ for $k=1,2,\ldots, N$ represent solutions to zero sum problems, where $N=\#(S)$._

There is a distinction between zero sums and zero subset sums. If $S$ has cardinality $\#(S)=N$, then we allow any element $s\in S$ to be used at most $N$ times in the polynomial formulation of the zero sum problem. This is in contrast to the zero subset sum problem, where an element $s\in S$ can occur _at most once_. For example $S=\{-2, 1, 3\}$ does not have any strict zero subset sums. However the constant coefficient in $q:=p^3=(x^{-2}+x+x^3)^3$ is nonzero because $-2+1+1=0$. So when we use polynomials to find zero sums, we must slightly expand the statement of the zero subset sum problem to allow for small multiplicity. A priori this small multiplicity is at most the cardinality of the set $N=\#(S)$.

In the zero subset sum problem, we have nothing except brute-force search over the subsets. There are no shortcuts or hidden information to accelerate the procedure. This same problem arises with the polynomials. For we begin with an initial set $S$ with Laurent polynomial $p=p_S$. However we need to evaluate the coefficients of powers $q=p^k$ of this polynomial.
_But the real issue is that the powers of $p$ are not readily computed a priori. We cannot decide whether the constant coefficient is zero or not zero simply from the definition of $q=p^k$._ This can only be decided _a posteriori_ after explicit computatation.

As obvious as this last sentence sounds, it's somewhat an open problem for most "logical mathematicians". 

Question: Are there any analytical tricks for finding the coefficients of $q=p^k$ ? We know none, except Cauchy Residue formula, but it's not clear this is useful in our case...



In [15]:
# How to find zero sums using Laurent polynomials and symbolic calculator:
from sympy import symbols, expand 

# the indeterminate x.
x = symbols('x') 

# formal Laurent polynomial defined by the subset S.
def poly(S): 
  return sum([x**k for k in S])

# example subsets.
S = [-1, -7, 17, 3, 13]; p = poly(S); print("The subset S =", S, "generates polynomial p =", p)
S1 = [-2, 1, 3]; p1 = poly(S1); print("The subset S1 =", S1, "generates polynomial p1 =", p1)
S2 = [-12, 25, -8, 20, -9, 6]; p2 = poly(S2); print("The subset S2 =", S2, "generates polynomial p2 =", p2)
S3 = [-3, 12, -8, 21, -19, 6]; p3 = poly(S3); print("The subset S3 =", S3, "generates polynomial p3 =", p3)

# returns the constant coefficient for small powers of p=poly(S):
def cc(S):
  p=poly(S); N = len(S); output=[];
  for j in range(N):
    q=expand(p**(j+1))
    output = output + [q.coeff(x,0)]
  #print("The constant coefficients of the small powers of p(S) are:")
  return output

# returns Boolean True or False depending on whether there exists zero sum in S. 
def existsZeroSum(S):
  val = False;
  if sum(cc(S))>0:
    val = True
  else:
    pass
  return val

The subset S = [-1, -7, 17, 3, 13] generates polynomial p = x**17 + x**13 + x**3 + 1/x + x**(-7)
The subset S1 = [-2, 1, 3] generates polynomial p1 = x**3 + x + x**(-2)
The subset S2 = [-12, 25, -8, 20, -9, 6] generates polynomial p2 = x**25 + x**20 + x**6 + x**(-8) + x**(-9) + x**(-12)
The subset S3 = [-3, 12, -8, 21, -19, 6] generates polynomial p3 = x**21 + x**12 + x**6 + x**(-3) + x**(-8) + x**(-19)


In [18]:
S0= [ 16, 4, 5, -17];
print(cc(S0)); print(existsZeroSum(S0))

[0, 0, 0, 0]
False


# [Incomplete]


# hide

So the idea is simply looking at powers $p^k$, for $k=1,2,3,\ldots$ of the Laurent polynomial $p=p_S$ generated by a finite subset $S$, and specifically evaluating the coefficient of the degree zero term. 

Problem: Given a Laurent polynomial $p \in \mathbf{Z}[x, x^{-1}]$, determine the degree zero coefficient of $p^k$ for large powers $k>>0$ and decide whether the degree zero coefficient is zero or nonzero.

The basic trick is:
_To compute solutions of zero sum problems we need determine the constant term of Laurent polynomials, and decide whether the constant term is zero or nonzero for powers $p^k$ of $p=p_S$. 

There is an idea we learned from Wolfram Mathematica and Prof. Dror Bar-Natan's course in Toronto called "Shameless Mathematica". The most interesting aspect of the course was Dror's presentation of the Khovanov homology as computed categorically from cubes and reductions, and it's actually a beautiful generalization of the Euler formula for computing Euler characteristics. It was excellent course which I used throughout my thesis, and which motivated the following ideas. 

_Remarks in the python code:_

Remark. We know that for large powers of p, there exist zero sums. This is linear algebra depending on the linear dependancies of S over integers.

Remark. Our evaluations involve a loss of information, for when we know that there exists zero sums  we do not necessarily know which 'subset' was actually summed. The computation somewhat becomes nonconstructive! All we know is that there *are* such subsets (and not even strictly subsets).

After inspection we see that [-3, -3, 6] is not a strict subset, but which sums to zero.
categorically/algebraically: how to restrict ourselves to distinct subsets??